<a href="https://colab.research.google.com/github/Decoding-Data-Science/airesidency/blob/main/Cohort_6_PineconeIndexDemo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<a href="https://colab.research.google.com/github/run-llama/llama_index/blob/main/docs/docs/examples/vector_stores/PineconeIndexDemo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Pinecone Vector Store

If you're opening this Notebook on colab, you will probably need to install LlamaIndex 🦙.

In [1]:
%pip install llama-index llama-index-vector-stores-pinecone

In [2]:
import logging
import sys
import os

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

#### Creating a Pinecone Index

In [3]:
from pinecone import Pinecone, ServerlessSpec

In [4]:
import openai
from google.colab import userdata

# Retrieve the OpenAI API key from Google Colab secrets
openai.api_key = userdata.get('openai')

if openai.api_key:
    os.environ["OPENAI_API_KEY"] = openai.api_key

In [5]:
os.environ["PINECONE_API_KEY"] = "pcsk_3hG9JF_HUuVnMGYDVoeTZctGjvTJRASUnQQjnDVFNUMX2ntkdBsfhLbbYxapdLzTaedRaK"


api_key = os.environ["PINECONE_API_KEY"]

pc = Pinecone(api_key=api_key)

In [6]:
#delete if needed
pc.delete_index("quickstart")

In [7]:
# dimensions are for text-embedding-ada-002

pc.create_index(
    name="quickstart",
    dimension=1536,
    metric="euclidean",
    spec=ServerlessSpec(cloud="aws", region="us-east-1"),
)


{
    "name": "quickstart",
    "metric": "euclidean",
    "host": "quickstart-0blulnl.svc.aped-4627-b74a.pinecone.io",
    "spec": {
        "serverless": {
            "cloud": "aws",
            "region": "us-east-1"
        }
    },
    "status": {
        "ready": true,
        "state": "Ready"
    },
    "vector_type": "dense",
    "dimension": 1536,
    "deletion_protection": "disabled",
    "tags": null
}

In [8]:
pinecone_index = pc.Index("quickstart")

#### Load documents, build the PineconeVectorStore and VectorStoreIndex

In [9]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.vector_stores.pinecone import PineconeVectorStore
from IPython.display import Markdown, display

Download Data

In [10]:
!mkdir -p 'data/paul_graham/'
!wget 'https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/paul_graham/paul_graham_essay.txt' -O 'data/paul_graham/paul_graham_essay.txt'

--2025-08-16 10:56:16--  https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/paul_graham/paul_graham_essay.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 75042 (73K) [text/plain]
Saving to: ‘data/paul_graham/paul_graham_essay.txt’

data/paul_graham/pa 100%[===================>]  73.28K  --.-KB/s    in 0.02s   

2025-08-16 10:56:17 (3.10 MB/s) - ‘data/paul_graham/paul_graham_essay.txt’ saved [75042/75042]



In [11]:
# load documents
documents = SimpleDirectoryReader("./data/paul_graham").load_data()

In [ ]:
documents

In [12]:
# initialize without metadata filter
from llama_index.core import StorageContext

vector_store = PineconeVectorStore(pinecone_index=pinecone_index)
storage_context = StorageContext.from_defaults(vector_store=vector_store)
index = VectorStoreIndex.from_documents(
    documents, storage_context=storage_context
)

Upserted vectors:   0%|          | 0/22 [00:00<?, ?it/s]

#### Query Index

May take a minute or so for the index to be ready!

In [16]:
# set Logging to DEBUG for more detailed outputs
query_engine = index.as_query_engine()
response = query_engine.query("what is this document about")
display(Markdown(f"<b>{response}</b>"))

<b>The document is about the experiences and reflections of the author, including insights into his writing process, career choices, interactions with the online community, and the evolution of certain industries like software and venture capital. It also touches on the challenges and dynamics of publishing essays, the impact of rapid change on various fields, and personal anecdotes related to his ventures and collaborations.</b>

In [14]:
display(Markdown(f"<b>{response}</b>"))

<b>The document is about the experiences and reflections of the author, including insights into his writing process, the evolution of technology, his involvement in startups like Y Combinator, and the challenges faced in fields affected by rapid change.</b>

In [17]:
pip install gradio

In [19]:
# Install necessary packages
#%pip install -q llama-index llama-index-vector-stores-pinecone gradio

# --- Imports ---
import os
import logging
import sys
import gradio as gr
from IPython.display import Markdown, display

from pinecone import Pinecone, ServerlessSpec
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader, StorageContext
from llama_index.vector_stores.pinecone import PineconeVectorStore

# --- Logging ---
logging.basicConfig(stream=sys.stdout, level=logging.INFO)

# --- Set API Keys ---

PINECONE_API_KEY = userdata.get("PINECONE_API_KEY")



api_key = os.environ["PINECONE_API_KEY"]


# --- Initialize Pinecone ---
pc = Pinecone(api_key=api_key)
index_name = "quickstart"
dimension = 1536

# Delete index if exists (optional)
if index_name in [idx['name'] for idx in pc.list_indexes()]:
    pc.delete_index(index_name)

# Create Pinecone index
pc.create_index(
    name=index_name,
    dimension=dimension,
    metric="euclidean",
    spec=ServerlessSpec(cloud="aws", region="us-east-1"),
)

pinecone_index = pc.Index(index_name)

# --- Load Data ---
!mkdir -p 'data/paul_graham/'
!wget -q 'https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/paul_graham/paul_graham_essay.txt' -O 'data/paul_graham/paul_graham_essay.txt'

#change directory
documents = SimpleDirectoryReader("./data/paul_graham").load_data()

# --- Create Index ---
vector_store = PineconeVectorStore(pinecone_index=pinecone_index)
storage_context = StorageContext.from_defaults(vector_store=vector_store)
index = VectorStoreIndex.from_documents(documents, storage_context=storage_context)

# --- Query Engine ---
query_engine = index.as_query_engine()

# --- Gradio App ---
def query_doc(prompt):
    try:
        response = query_engine.query(prompt)
        return str(response)
    except Exception as e:
        return f"Error: {str(e)}"

gr.Interface(
    fn=query_doc,
    inputs=gr.Textbox(label="Ask a question about the document"),
    outputs=gr.Textbox(label="Answer"),
    title=" Document QA (LlamaIndex + Pinecone)",
    description="Ask questions based on the indexed Social Media Regualtion. Powered by LlamaIndex & Pinecone."
).launch(share=True)


Upserted vectors:   0%|          | 0/22 [00:00<?, ?it/s]

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://b3ecb322fc2c76a843.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
